In [1]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers
from scipy.stats import rankdata
from sklearn.metrics import r2_score
import textdistance as TD
from multiprocessing import Pool
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
import dill

In [2]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
        "build-linux-kernel1",
        "fio2",
        "fio3",
        "fio4",
        "fio5",
        "fio6",
        "fio7",
        "fio8",
        "fio9",
        "iperf10",
        "iperf11",
        "iperf12",
        "iperf13",
        "john-the-ripper14",
        "john-the-ripper15",
        "ramspeed16",
        "ramspeed17",
        "ramspeed18",
        "ramspeed19",
        "ramspeed20",
        "ramspeed21",
        "ramspeed22",
        "ramspeed23",
        "ramspeed24",
        "ramspeed25",
        "stream26",
        "stream27",
        "stream28",
        "stream29",
        "pCpu",
        "cpus",
        "rss",
        "vmem",
        "rchar",
        "wchar",
        "syscr",
        "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                   'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                   'syscw']
x_cols = all_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

In [3]:
from my_yaml import yaml_load

with open("nodeConfigIdLookup.yaml", "r") as f:
    t = yaml_load(f)
    nodeIDLUT = {v: k for k, v in t.items()}
dF["nodeName"] = dF["nodeConfig"].transform(lambda x: nodeIDLUT[x])

In [4]:
awsGroupLUT = {
        "general": ["M7g", "Mac", "M6g", "M6i", "M6in", "M6a", "M5", "M5n", "M5zn", "M5a", "M4", "A1", "T4g", "T3",
                    "T3a",
                    "T2"],
        "compute": ["C7g", "C7gn", "C6i", "C6in", "C6a", "C6g", "C6gn", "C5", "C5n", "C5a", "C4"],
        "memory": ["R7g", "R7iz", "R6g", "R6i", "R6in", "R6a", "R5", "R5n", "R5b", "R5a", "R4", "X2gd", "X2idn",
                   "X2iedn",
                   "X2iezn", "X1", "X1e", "z1d"],
        "accelerated": ["P4", "P3", "P2", "DL1", "Trn1", "Inf2", "Inf1", "G5", "G5g", "G4dn", "G4ad", "G3", "F1",
                        "VT1"],
        "storage": ["Im4gn", "Is4gen", "I4i", "I3", "I3en", "D2", "D3", "D3en", "H1"],
        "HPC": ["Hpc6id", "Hpc6a"]
}

In [5]:
awsInstanceGroupLUT = {i.lower(): k for k, v in awsGroupLUT.items() for i in v}

In [6]:
dF["nodeGroup"] = dF["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [7]:
usedAwsGroupLUT = {
        k: list(pds.unique(dF.query("nodeGroup==@k").nodeName.values)) for k in awsGroupLUT.keys()
}

In [8]:
nodesPerGroup = {
        k: len(v) for k, v in usedAwsGroupLUT.items()
}

In [9]:
import seaborn as sns

if False:
    #sns.boxplot(data=dF,y="realtime",x="taskName",hue="nodeGroup")
    for wf in dF.groupby(by="wfName"):
        wfname, data = wf
        print(wfname)
        grid = sns.FacetGrid(data, row="wfName", col="nodeGroup", sharex=False, sharey=True, margin_titles=True,
                             height=5)
        grid.map_dataframe(sns.boxplot, y="realtime", x="taskName")

# Find prototypes for each group

In [10]:
true_ranks_df = pds.DataFrame()
for l, d in dF.groupby(by=["wfName", "taskName", "nodeName"]):
    wn, tn, nn = l
    true_ranks_df = true_ranks_df.append(
            {"wfName": wn, "taskName": tn, "nodeName": nn, "realtime": d.realtime.mean(), "rank": 0},
            ignore_index=True)

for l, d in true_ranks_df.groupby(by=["wfName", "taskName"]):
    wn, tn = l
    ranks = rankdata(d.realtime.values, method="min")
    d["rank"] = ranks
    true_ranks_df.update(d)

In [11]:
with db_actions.connect() as conn:
    scores_df = pds.read_sql('SELECT * FROM "nodeBenchmarkTransposedScores"', conn)


In [12]:
scores_df["nodeName"] = scores_df["nodeConfig"].transform(lambda x: nodeIDLUT[x])
scores_df["nodeGroup"] = scores_df["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [13]:
groupOrder = ["general", "compute", "memory", "storage"]
nodeOrder = list(pds.unique(dF.nodeName))

In [14]:
benchmarkCols = [
        "build-linux-kernel1",
        "fio2",
        "fio3",
        "fio4",
        "fio5",
        "fio6",
        "fio7",
        "fio8",
        "fio9",
        "iperf10",
        "iperf11",
        "iperf12",
        "iperf13",
        "john-the-ripper14",
        "john-the-ripper15",
        "ramspeed16",
        "ramspeed17",
        "ramspeed18",
        "ramspeed19",
        "ramspeed20",
        "ramspeed21",
        "ramspeed22",
        "ramspeed23",
        "ramspeed24",
        "ramspeed25",
        "stream26",
        "stream27",
        "stream28",
        "stream29"]

In [15]:
def find_group_protos(benchmarkCols=benchmarkCols, scores_dF=scores_df, groupOrder=groupOrder):
    scores = dict()
    nodeGroups = [scores_dF.query("nodeGroup==@gn").nodeName.values for gn in groupOrder]
    nodeGroupBenchmarkAverages = [np.average([scores_dF.query("nodeName==@nn")[benchmarkCols] for nn in ng], axis=0) for
                                  ng
                                  in nodeGroups]
    for c in itertools.product(*nodeGroups):
        proto_benchmarks = [scores_dF.query("nodeName == @pn")[benchmarkCols].values[0] for pn in c]
        distance_between_protos = 0
        for i, pn1 in enumerate(proto_benchmarks[:-1]):
            for pn2 in proto_benchmarks[i + 1:]:
                distance_between_protos += np.linalg.norm(pn1 - pn2)
        distances_from_group_averages = [np.linalg.norm(bs - ngba) for bs, ngba in
                                         zip(proto_benchmarks, nodeGroupBenchmarkAverages)]
        scores[c] = (distance_between_protos, np.sum(distances_from_group_averages))
    factor = sum(x[0] for x in scores.values()) / sum(x[1] for x in scores.values())
    scores = {
            k: v[0] + factor * v[1] for k, v in scores.items()
    }
    cs, ds = scores.keys(), scores.values()
    return [x[0] for x in sorted(zip(cs, ds), key=lambda x: x[1])]

In [16]:
bestGroupPrototypes = (
        'm5.xlarge', 'c5.xlarge', 'r5.xlarge', 'i3.2xlarge')  # = sorted(zip(cs, ds), key=lambda x: x[1])[0][0]
bestGroupPrototypesLUT = {
        awsInstanceGroupLUT[k.split(".")[0]]: k for k in bestGroupPrototypes
}

In [17]:
print(bestGroupPrototypesLUT)

{'general': 'm5.xlarge', 'compute': 'c5.xlarge', 'memory': 'r5.xlarge', 'storage': 'i3.2xlarge'}


# Produce Group Rankings

In [18]:
def makeGroupRankings(prototypes, instanceGroupLUT=awsInstanceGroupLUT, base_dF=dF, groupOrder=groupOrder):
    try:
        prototypesLUT = {
                instanceGroupLUT[k.split(".")[0]]: k for k in prototypes
        }
    except:
        prototypesLUT = {
                instanceGroupLUT[k]: k for k in prototypes
        }
    groupPerformances = dict()
    for g in base_dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
        l, d = g
        ng, wfn, tn = l
        if (wfn, tn) not in groupPerformances.keys():
            groupPerformances[(wfn, tn)] = dict()
        groupProto = prototypesLUT[ng]
        groupPerformances[(wfn, tn)][ng] = d.query("nodeName == @groupProto").realtime.mean()
        #groupPerformances[l] = d.realtime.mean()
    groupRankings = {"wfName": [], "taskName": [], **{o: [] for o in groupOrder}}
    for k, v in groupPerformances.items():
        wfn, tn = k
        performances = [v[i] for i in groupOrder]
        ranks = rankdata(performances, method="min")
        groupRankings["wfName"].append(wfn)
        groupRankings["taskName"].append(tn)
        for o, r in zip(groupOrder, ranks):
            groupRankings[o].append(r)
    return pds.DataFrame(groupRankings)


#### Produce intragroup node rankings

In [19]:
def makeNodeRankingsInGroups(groupRankings, base_dF=dF, groupOrder=groupOrder, nodesPerGroup=nodesPerGroup):
    tmp = pds.DataFrame(
            columns=["group", "nodeName", "wfName", "taskName", "realtime", "groupRank", "intraGroupRank", "globalRank",
                     "trueRank"])
    tmp2 = {"group": [], "nodeName": [], "wfName": [], "taskName": [], "realtime": []}
    for g in base_dF.groupby(by=["nodeGroup", "wfName", "taskName", "nodeName"]):
        l, d = g
        ng, wfn, tn, nn = l
        tmp2["group"].append(ng)
        tmp2["nodeName"].append(nn)
        tmp2["wfName"].append(wfn)
        tmp2["taskName"].append(tn)
        tmp2["realtime"].append(d.realtime.mean())
    tmp = tmp.append(pds.DataFrame(tmp2), ignore_index=True)
    del tmp2

    for l, d in tmp.groupby(["group", "wfName", "taskName"]):
        gr, wfn, tn = l
        ra = list(rankdata(d.realtime, method="min"))
        cache = groupRankings.query("wfName==@wfn and taskName==@tn")
        ar = 0
        grr = cache[gr].values[0]
        for k, v in sorted(list(zip(groupOrder, (cache[k].values[0] for k in groupOrder))), key=lambda x: x[1]):
            if v >= grr:
                break
            ar += nodesPerGroup[k]
        # d["intraGroupRank"] = ra
        # d["groupRank"] = [grr] * len(ra)
        # d["globalRank"] = np.array([ar] * len(ra)) + np.array(ra)
        tmp.update(pds.DataFrame({"intraGroupRank": ra, "groupRank": [grr] * len(ra),
                                  "globalRank": np.array([ar] * len(ra)) + np.array(ra)}, index=d.index))

    for l, d in tmp.groupby(["wfName", "taskName"]):
        tr = list(rankdata(d.realtime, method="min"))
        #d["trueRank"] = tr
        tmp.update(pds.DataFrame({"trueRank": tr}, index=d.index))

    return tmp

In [ ]:
tmp3 = makeNodeRankingsInGroups(makeGroupRankings(bestGroupPrototypes))

In [ ]:
export = pds.DataFrame(
        {"wfName": tmp3.wfName, "taskName": tmp3.taskName, "nodeName": tmp3.nodeName, "rank": tmp3.globalRank})

In [ ]:
export.to_csv("no_ML_ranks.csv", index=False)

In [20]:
def scoreRankings(ranking_df):
    scores = []
    for l, d in ranking_df.groupby(by=["wfName", "taskName"]):
        scores.append(commons.rankingScore(d.globalRank.values, d.trueRank.values))
    return np.average(scores), np.std(scores)

In [ ]:
print(scoreRankings(tmp3))

In [ ]:
for l, d in tmp3.groupby("wfName"):
    print(l, scoreRankings(d))

# Bruteforce test all prototype selections

In [ ]:
def do_one(x):
    return x, scoreRankings(makeNodeRankingsInGroups(makeGroupRankings(x)))


nodeGroups = [scores_dF.query("nodeGroup==@gn").nodeName.values for gn in groupOrder]

with Pool() as pool:
    results = pool.map(do_one, itertools.product(*nodeGroups))
    prototypeSelectionScores = {c: r for c, r in results}

In [ ]:
print(sorted(zip(prototypeSelectionScores.keys(), prototypeSelectionScores.values()),
             key=lambda x: prototypeSelectionScores[x[0]][0], reverse=True)[:3])

# New Instance Type Clusterings

In [21]:
def make_clustered_df(instanceGroupLUT, base_dF=dF, scores_dF=scores_df):
    clustered_dF = base_dF.copy()
    clustered_dF["nodeGroup"] = clustered_dF["nodeName"].transform(lambda x: instanceGroupLUT[x])
    clustered_groupOrder = pds.unique(clustered_dF.nodeGroup)
    clustered_scores_df = scores_dF.copy()
    clustered_scores_df["nodeGroup"] = clustered_scores_df["nodeName"].transform(lambda x: instanceGroupLUT[x])
    return clustered_dF, clustered_groupOrder, clustered_scores_df

In [22]:
def make_clustered_ranking(instanceGroupLUT=awsInstanceGroupLUT, protos=None, clustered_dF=None,
                           clustered_groupOrder=None, clustered_scores_df=None):
    if clustered_dF is None or clustered_groupOrder is None or clustered_scores_df is None:
        clustered_dF, clustered_groupOrder, clustered_scores_df = make_clustered_df(instanceGroupLUT)
    if protos is None:
        bestClusteredGroupPrototypes = \
            find_group_protos(scores_dF=clustered_scores_df, groupOrder=clustered_groupOrder)[0]
    else:
        bestClusteredGroupPrototypes = protos
    #print(bestClusteredGroupPrototypes)
    return makeNodeRankingsInGroups(
            makeGroupRankings(prototypes=bestClusteredGroupPrototypes, instanceGroupLUT=instanceGroupLUT,
                              base_dF=clustered_dF, groupOrder=clustered_groupOrder),
            base_dF=clustered_dF,
            groupOrder=clustered_groupOrder,
            nodesPerGroup={l: len(d.unique()) for l, d in clustered_dF.groupby("nodeGroup").nodeName})

In [23]:
def score_clustering(instanceGroupLUT=awsInstanceGroupLUT):
    return scoreRankings(make_clustered_ranking(instanceGroupLUT=instanceGroupLUT))

In [ ]:
instanceGroupLUT = {nn: c for nn, c in zip(scores_df.nodeName.values,
                                           AffinityPropagation(random_state=None).fit_predict(
                                                   scores_df[benchmarkCols]))}
print(score_clustering(instanceGroupLUT))

In [ ]:
for n_clusters in range(2, 11):
    print(n_clusters, score_clustering({nn: c for nn, c in zip(scores_df.nodeName.values,
                                                               KMeans(n_clusters).fit_predict(
                                                                       scores_df[benchmarkCols]))}
                                       ))

In [24]:
tmp3 = make_clustered_ranking({nn: c for nn, c in zip(scores_df.nodeName.values,
                                                      KMeans(n_clusters=4) \
                                                      #AffinityPropagation(random_state=None)
                                                      .fit_predict(
                                                              scores_df[benchmarkCols]))}
                              )
print(scoreRankings(tmp3))
for l, d in tmp3.groupby("wfName"):
    print(l, scoreRankings(d))

(0.8293656995016583, 0.04422357750589898)
nfcore/chipseq:1.2.2 (0.8139917695473251, 0.05303943362894737)
nfcore/eager:2.3.5 (0.8168724279835391, 0.04751359056822495)
nfcore/methylseq:1.6.1 (0.850632525529645, 0.05113180636186376)
nfcore/sarek:2.7.1 (0.850632525529645, 0.019274421532738582)
nfcore/viralrecon:2.1 (0.8301118177661387, 0.034452177038331415)


In [34]:
yo = list()
for l, d in tmp3.query("wfName=='nfcore/chipseq:1.2.2'").groupby("taskName"):
    yo.append((l, scoreRankings(d)[0], d.realtime.mean()))
print("\n".join(repr(x) for x in sorted(yo, key=lambda x:x[2], reverse=True)))

('PLOTPROFILE', 0.868312757201646, 177618.21975308642)
('PLOTFINGERPRINT', 0.747599451303155, 21291.020370370374)
('TRIMGALORE', 0.8628257887517147, 15768.949382716048)
('CONSENSUS_PEAKS_DESEQ2', 0.8161865569272977, 15197.34074074074)
('MERGED_BAM', 0.8271604938271605, 9175.264197530863)
('BWA_INDEX', 0.7613168724279835, 8716.503703703705)
('FASTQC', 0.8573388203017833, 8591.530864197532)
('BWA_MEM', 0.8655692729766804, 6214.077777777778)
('PICARD_METRICS', 0.8244170096021948, 5661.207407407408)
('PHANTOMPEAKQUALTOOLS', 0.7914951989026063, 5514.793827160494)
('MULTIQC', 0.8765432098765432, 5424.859259259259)
('get_software_versions', 0.879286694101509, 5127.977777777777)
('MACS2_ANNOTATE', 0.747599451303155, 5028.7759259259265)
('CONSENSUS_PEAKS_ANNOTATE', 0.7311385459533608, 5009.281481481481)
('MACS2_QC', 0.7338820301783264, 4648.7555555555555)
('PRESEQ', 0.8244170096021948, 3439.2759259259265)
('MERGED_BAM_FILTER', 0.887517146776406, 3015.0395061728395)
('MACS2', 0.821673525377229, 

In [36]:
yo = list()
for l, d in tmp3.query("wfName=='nfcore/viralrecon:2.1'").groupby("taskName"):
    yo.append((l, scoreRankings(d)[0], d.realtime.mean()))
print("\n".join(repr(x) for x in sorted(yo, key=lambda x:x[2], reverse=True)))

('NFCORE_VIRALRECON:ILLUMINA:ASSEMBLY_UNICYCLER:UNICYCLER', 0.8628257887517147, 144293.82716049382)
('NFCORE_VIRALRECON:ILLUMINA:ALIGN_BOWTIE2:BOWTIE2_ALIGN', 0.8710562414266119, 50071.60493827161)
('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:ASCIIGENOME', 0.8573388203017833, 41716.049382716046)
('NFCORE_VIRALRECON:ILLUMINA:ASSEMBLY_SPADES:SPADES', 0.823045267489712, 30617.28395061729)
('NFCORE_VIRALRECON:ILLUMINA:MULTIQC', 0.8628257887517147, 23815.17777777777)
('NFCORE_VIRALRECON:ILLUMINA:CUTADAPT', 0.840877914951989, 13017.283950617284)
('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_BCFTOOLS:ASCIIGENOME', 0.9039780521262003, 11190.123456790121)
('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:PANGOLIN', 0.8189300411522633, 8241.975308641975)
('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_BCFTOOLS:PANGOLIN', 0.8326474622770919, 8019.753086419754)
('NFCORE_VIRALRECON:ILLUMINA:ASSEMBLY_SPADES:ASSEMBLY_QC:QUAST', 0.7722908093278463, 7019.34814814815)
('NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:IVAR_VARIANTS', 0.8

In [ ]:
export = pds.DataFrame(
        {"wfName": tmp3.wfName, "taskName": tmp3.taskName, "nodeName": tmp3.nodeName, "rank": tmp3.globalRank})
export.to_csv("no_ML_ranks.csv", index=False)

In [30]:
class RunParams(NamedTuple):
    protos: Any
    instanceGroupLUT: Any


clustered_dF, clustered_groupOrder, clustered_scores_df = None, None, None


def do_one(inp: str):
    x: RunParams = dill.loads(inp)
    return inp, scoreRankings(make_clustered_ranking(x.instanceGroupLUT, x.protos, clustered_dF, clustered_groupOrder,
                                                     clustered_scores_df))


def search_protos_for_clustering(instanceGroupLUT):
    global clustered_dF, clustered_groupOrder, clustered_scores_df
    clustered_dF, clustered_groupOrder, clustered_scores_df = make_clustered_df(instanceGroupLUT)
    nodeGroups = [clustered_scores_df.query("nodeGroup==@gn").nodeName.values for gn in clustered_groupOrder]

    with Pool() as pool:
        results = pool.map(do_one,
                           (dill.dumps(
                                   RunParams(c, instanceGroupLUT))
                                   for
                                   c in itertools.product(*nodeGroups)))
        prototypeSelectionScores = {dill.loads(c).protos: r for c, r in results}
    return sorted(zip(prototypeSelectionScores.keys(), prototypeSelectionScores.values()),
                  key=lambda x: prototypeSelectionScores[x[0]][0], reverse=True)

In [31]:
res = search_protos_for_clustering({nn: c for nn, c in zip(scores_df.nodeName.values,
                                                           KMeans(n_clusters=4).fit_predict(
                                                                   scores_df[benchmarkCols]))})

In [49]:
c = 0
for r in itertools.takewhile(lambda x: x[1][0] > 0.8293656995016583, res):
    print(r)
    c += 1
print(c)

(('m5zn.2xlarge', 'm5.2xlarge', 'r5a.large', 'd3.xlarge'), (0.8350610338420933, 0.03586456592654338))
(('m5zn.2xlarge', 'r5.2xlarge', 'r5a.large', 'd3.xlarge'), (0.8342449341042874, 0.03596220286241507))
(('z1d.2xlarge', 'm5.2xlarge', 'r5a.large', 'd3.xlarge'), (0.83421020645587, 0.036599218069512315))
(('c5.2xlarge', 'm5.2xlarge', 'r5a.large', 'd3.xlarge'), (0.8342102064558697, 0.037202097711907446))
(('z1d.2xlarge', 'r5.2xlarge', 'r5a.large', 'd3.xlarge'), (0.8333941067180638, 0.036769372734758735))
(('c5.2xlarge', 'r5.2xlarge', 'r5a.large', 'd3.xlarge'), (0.8332204684759772, 0.03730623187485074))
(('m5zn.2xlarge', 'm5.2xlarge', 'i3.large', 'd3.xlarge'), (0.8331510131791425, 0.03769783411074239))
(('m5zn.2xlarge', 'd3.2xlarge', 'r5a.large', 'd3.xlarge'), (0.8330468302338907, 0.03582997251505765))
(('m5zn.2xlarge', 'r5.2xlarge', 'i3.large', 'd3.xlarge'), (0.8329773749370563, 0.03892266871534295))
(('c5.2xlarge', 'm5.2xlarge', 'i3.large', 'd3.xlarge'), (0.8326127346286747, 0.0381424214

In [53]:
res = search_protos_for_clustering({nn: c for nn, c in zip(scores_df.nodeName.values,
                                                           AffinityPropagation(random_state=None).fit_predict(
                                                                   scores_df[benchmarkCols]))})

In [54]:
c = 0
for r in itertools.takewhile(lambda x: x[1][0] > 0.7407233769165323, res):
    print(r)
    c += 1
print(c)

(('m5zn.2xlarge', 'z1d.large', 'm5zn.xlarge', 'r5a.large'), (0.7836467503602995, 0.05429187207880372))
(('m5zn.2xlarge', 'z1d.large', 'm5zn.xlarge', 'm5a.large'), (0.7830042888645796, 0.05465485367695197))
(('m5zn.2xlarge', 'z1d.large', 'm5zn.xlarge', 'i3.large'), (0.7828480144467018, 0.05427143186080806))
(('m5zn.2xlarge', 'z1d.large', 'm5zn.xlarge', 'm5a.xlarge'), (0.7824312826656943, 0.05472816816821849))
(('m5zn.2xlarge', 'z1d.large', 'm5zn.xlarge', 'r5a.xlarge'), (0.7824312826656943, 0.05472816816821849))
(('z1d.2xlarge', 'r5.large', 'z1d.xlarge', 'i3.large'), (0.7815978191036793, 0.053639413366760015))
(('z1d.2xlarge', 'r5.large', 'z1d.xlarge', 'r5a.large'), (0.7809727214321681, 0.05450457676518012))
(('z1d.2xlarge', 'm5.large', 'z1d.xlarge', 'r5a.large'), (0.7809032661353335, 0.05390751286231339))
(('z1d.2xlarge', 'm5.large', 'z1d.xlarge', 'i3.large'), (0.7808859023111249, 0.05337947402052992))
(('z1d.2xlarge', 'r5.large', 'z1d.xlarge', 'm5a.xlarge'), (0.7804865343543261, 0.0540